In [10]:
# specification of environnement (not use here below)
%%writefile conda_dependencies.yml

dependencies:
- python=3.6.2
- pip:
  - azureml-defaults
  - keras
  - numpy
  - scikit-learn
  - pandas
  - tensorflow<=2.4
  - matplotlib
  - raiwidgets


Overwriting conda_dependencies.yml


In [ ]:
#!pip install azureml-contrib-fairness
#!pip install fairlearn==0.4.6
#!pip install raiwidgets
#pip install azureml-dataprep[pandas]

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core import Environment
from azureml.core import *
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import RunConfiguration
from azureml.core import Workspace, Dataset, Datastore
import os, shutil
from azureml.core import Dataset
from azureml.pipeline.steps import PythonScriptStep
from azureml.pipeline.core import Pipeline, PipelineData
from azureml.core import Workspace, Dataset
from azureml.train.automl import AutoMLConfig
from azureml.core import Workspace,RunConfiguration
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.train.automl import AutoMLConfig
from azureml.pipeline.steps import AutoMLStep
from azureml.train.automl.utilities import get_primary_metrics
from azureml.data.datapath import DataPath
from azureml.core.datastore import Datastore
from azureml.pipeline.core import InputPortBinding
from azureml.core import Run
from azureml.data.dataset_consumption_config import DatasetConsumptionConfig
from azureml.pipeline.core import PipelineParameter
from azureml.core import Dataset
from azureml.core import Environment
from azureml.pipeline.core import PipelineData, TrainingOutput
from azureml.pipeline.steps import HyperDriveStep, HyperDriveStepRun, PythonScriptStep
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.parameter_expressions import choice
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.data import OutputFileDatasetConfig
from azureml.core import Experiment, ScriptRunConfig, Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.train.hyperdrive import *
from azureml.widgets import RunDetails
from azureml.pipeline.core.schedule import ScheduleRecurrence, Schedule
from azureml.core import Run
from azureml.core import get_run
import pandas as pd
import numpy as np

In [ ]:
# Environnement
subscription_id = '---------------'
resource_group = 'learning'
workspace_name = 'training_MLservices'
compute_engine = 'jcharley4'
cluster_name = 'jcharley4'
dataset_name = 'trainingdataset'
experiment_name = "test1208_v2"

ws = Workspace.from_config()
ws.get_details()

In [ ]:
# Environnement Creation

# Set the directory for the experiment files 
components_dir = '.'
os.makedirs(components_dir, exist_ok=True)

# Create a directory inside for the register component
os.makedirs(os.path.join(components_dir, "register"), exist_ok=True)

# Verify that the compute cluster exists
try:
    # Check for existing compute target
    pipeline_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # If it doesn't already exist, create it
    try:
        compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2', max_nodes=3)
        pipeline_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
        pipeline_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)

# Create a Python environment for the pipeline experiment
#pipeline_environment = Environment.from_conda_specification(name = 'pipeline-env', 
                                                           #file_path = os.path.join(components_dir, 
                                                           #"conda_dependencies.yml"))
#pipeline_environment.python.user_managed_dependencies = True # Let Azure ML manage dependencies
#pipeline_environment.docker.enabled = True # Use a docker container

# Create a new runconfig object for the pipeline
pipeline_run_config = RunConfiguration()

# Use the compute you created above.
pipeline_run_config.target = pipeline_cluster

# Assign the environment to the run configuration
#pipeline_run_config.environment = pipeline_environment
sklearn_env = Environment("sklearn-env")
#myenv = Environment.from_conda_specification(name = 'sklearn-env', file_path = './conda_dependencies.yml')

print ("Pipeline configuration created.")

Found existing cluster, use it.
Pipeline configuration created.


In [ ]:
# Step 0 environnement
data_folder = os.path.join(os.getcwd(), 'data/')
os.makedirs(data_folder, exist_ok=True)

# Step 0 data ingestion
raw_ds = Dataset.get_by_name(ws, name=dataset_name, version='latest')
datastore = ws.get_default_datastore()
# allow to use output1 in input2
fileConfig = OutputFileDatasetConfig(name='file_dataset')

sklearn_env = Environment("sklearn-env")
packages = CondaDependencies.create(conda_packages=['scikit-learn','pip'],
                                    pip_packages=['azureml-defaults'])
sklearn_env.python.conda_dependencies = packages

# Step 1 data preparation
step1 = PythonScriptStep(name = 'prepare data',
                         source_directory = 'scripts',
                         script_name = 'data_prep.py',
                         compute_target = 'jcharley3',
                         allow_reuse = True,
                         arguments = ['--raw-ds', raw_ds.as_named_input('raw_data'),
                                      '--output-dir', fileConfig])


# Step 2 : training 
train_src = ScriptRunConfig(source_directory='scripts',
                            script='model_script2.py',
                            compute_target= compute_engine,
                            arguments = ["--input-dir", fileConfig.as_input()],
                            environment=sklearn_env)


# Step 2 : hyperparameter 
param_sampling = RandomParameterSampling( {
    "--n_estimators": choice(15, 50, 100, 200, 300),
    "--criterion": choice("gini", "entropy"),
    "--max_depth": choice(2, 8, 16)
    }
)

hd_config = HyperDriveConfig(run_config=train_src, 
                             hyperparameter_sampling=param_sampling,
                             primary_metric_name='precision', 
                             primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, 
                             max_total_runs=3, # 1 for testing 
                             max_concurrent_runs=4)


saved_model = PipelineData(name='saved_model',
                           datastore=datastore,
                           pipeline_output_name='model_output',
                           training_output=TrainingOutput(type = "Model", metric="Precision", model_file="outputs/model/save_model.pkl"))

metrics_data = PipelineData(name='metrics_data', 
                            datastore=datastore,
                            pipeline_output_name='metrics_output',
                            training_output=TrainingOutput(type='Metrics'))

                                
 
hd_step = HyperDriveStep(
    allow_reuse=True,
    name='hyperparameters',
    outputs = [metrics_data, saved_model],
    hyperdrive_config=hd_config,
                             )



# step 3 : register best model
# best model is retrieve directly from PipelineData saved model
register_model = PythonScriptStep(name = 'Model Registration',
                                  script_name = 'scripts/register_model.py',
                                  inputs = [saved_model],
                                  compute_target = compute_engine,
                                  allow_reuse=True,
                                  arguments = ["--saved-model", saved_model,
                                                "--model-file", "model_sab.pkl",
                                                "--model-description","pipeline with hyperameters",
                                                "--model-name","best_model_1208"]
                                  )



# step 3 : publish pipeline
pipeline = Pipeline(workspace=ws, steps=[step1, hd_step, register_model], description="test-pipeline_3")
pipeline_run = pipeline.submit(experiment_name, regenerate_outputs=True)

published_pipeline1 = pipeline.publish(
                        name="Template_Pipeline_Notebook",
                        description="Published Pipeline Description",
                        version="1.0")


# step 3 : schedule pipeline run every day
#recurrence = ScheduleRecurrence(frequency='Day', interval=1)
#recurring_schedule = Schedule.create(ws, name='DailySchedule', 
#                            description='Once a day',
#                            pipeline_id=published_pipeline1.id, 
#                            experiment_name= experiment_name, 
#                            recurrence=recurrence)


'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.
'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.


Created step prepare data [4838634f][81f1cfb5-3dc1-45bb-a15f-9726a69644c0], (This step will run and generate new outputs)Created step hyperparameters [245cd62d][2607e1d0-cd28-439a-9fa0-94ef6c550638], (This step will run and generate new outputs)

Created step Model Registration [d068a2a5][1a6a6626-22b1-4e14-8bbf-8dae21911034], (This step will run and generate new outputs)
Submitted PipelineRun 12f3b5b0-1bf0-46d8-a633-a8d9803f644a
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/12f3b5b0-1bf0-46d8-a633-a8d9803f644a?wsid=/subscriptions/a0f4cddc-a66a-4dcc-9df7-ccbd7f81bf7b/resourcegroups/learning/workspaces/training_MLservices&tid=72f988bf-86f1-41af-91ab-2d7cd011db47
